In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import example_utils as ex_utils

from MARBLE import utils, geometry, net, plotting, postprocessing

OSError: dlopen(/Users/arnaudon/base/lib/python3.10/site-packages/torch_sparse/_diag_cpu.so, 0x0006): Symbol not found: __ZN3c106detail19maybe_wrap_dim_slowExxb
  Referenced from: /Users/arnaudon/base/lib/python3.10/site-packages/torch_sparse/_diag_cpu.so
  Expected in: /Users/arnaudon/base/lib/python3.10/site-packages/torch/lib/libc10.dylib

# For initial conditions, sample a rectangle uniformly

In [ ]:
t0, t1, dt = 0, 3, 0.5
t = np.arange(t0, t1, dt)
n = 100
area = [[-3,-3],[3,3]]

# Geneate phase portraits from random initial conditions while varying $\beta_1$

In [ ]:
n_steps=20
mu = np.hstack((np.linspace(-.5,.5,n_steps)))

pos, vel = [], []
X0_range = ex_utils.initial_conditions(n, len(mu), area)
for i, m in enumerate(mu):
    p, v = ex_utils.simulate_vanderpol(m, X0_range[i], t)
            
    pos.append(np.vstack(p))
    vel.append(np.vstack(v))
    
#embed on parabola
for i, (p, v) in enumerate(zip(pos, vel)):
    end_point = p + v
    new_endpoint = ex_utils.parabola(end_point[:,0], end_point[:,1])
    pos[i] = ex_utils.parabola(p[:,0], p[:,1])
    vel[i] = new_endpoint - pos[i]

# Plot vector fields

In [ ]:
data = utils.construct_dataset(pos, features=vel, graph_type='cknn', k=20, n_geodesic_nb=30, stop_crit=0.03, vector=False)

In [ ]:
titles = [r'$\beta$ = {:0.2f}'.format(m) for m in mu]

axes = plotting.fields(data, col=4, alpha=1, width=7, scale=2, titles=titles, view=[70,20])
for ax in axes:
    ex_utils.plot_parabola(ax)

#plt.savefig('./outputs/parabolas.svg')

# Train network

In [ ]:
par = {'epochs': 50, #optimisation epochs
       'order': 2, #order of derivatives
       'hidden_channels': 32, #number of internal dimensions in MLP
       'out_channels': 5,
       'inner_product_features': True,
      }

model = net(data, par=par)
model.run_training(data, outdir='./output/vanderpol_data')
#model.load_model('./outputs/vanderpol_data/best_model.pth')

# Create embeddings

In [ ]:
data = model.evaluate(data)
data = postprocessing(data)

emb_MDS, _ = geometry.embed(data.dist, embed_typ = 'MDS')
plotting.embedding(emb_MDS, mu, s=30, alpha=1)
#plt.savefig('../results/van_der_pol_MDS.svg')
plt.show()

# Distance matrix

In [ ]:
im = plt.imshow(data.dist)
plt.colorbar(im)
#plt.savefig('../results/van_der_pol_D.svg')

# Cluster and visualise embedding

In [ ]:
titles = [r'$\beta$ = {:0.2f}'.format(m) for m in mu]
plotting.embedding(data, mu[data.y.numpy().astype(int)], titles=titles)
#plt.savefig('../results/van_der_pol_embedding.svg')